## Example of generating data and training the AA channel<br>


In [1]:
from multiprocessing import cpu_count
import os
os.environ["JULIA_NUM_THREADS"] = str(cpu_count())
from juliacall import Main as jl
jl.Pkg.activate("/home/zaldivar/Documents/Androniki/Github/GWBackFinder.jl")
jl.seval("using GWBackFinder")
from tqdm import tqdm
import sys
import numpy as np
sys.path.append("/home/zaldivar/Documents/Androniki/Github/GWBackFinder_python")
from src.GWBackFinder import train as GW_train
from src.GWBackFinder import prior as GW_prior
import torch

  Activating project at `~/Documents/Androniki/Github/GWBackFinder.jl`
[ Info: Precompiling GWBackFinder [7fa44c00-b40e-49bf-9bf3-4f6997ab6254]
/home/zaldivar/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



#### load the prior and sample 1 million points

In [2]:
custom_prior=GW_prior.get_prior()
z=custom_prior.sample(1000000)
#z=np.save("./samples.npy",z)
# %%

/home/zaldivar/.local/lib/python3.9/site-packages/sbi/utils/user_input_checks_utils.py:69: UserWarning: Prior is lacking mean attribute, estimating prior mean from samples.
  warnings.warn(
/home/zaldivar/.local/lib/python3.9/site-packages/sbi/utils/user_input_checks_utils.py:80: UserWarning: Prior is lacking variance attribute, estimating prior variance from
                samples...
  warnings.warn(


### Define frequency range and split in 26 bins of equal log-spacing. $idx_{26}$ shows in which bin the frequency belongs to and $logbins_{26}$ which are the boundaries of the bins. After we coarse grain the frequencies from $3 \cdot 10^{-3}$ Hz to the maximum frequency 0.5 Hz, i.e., we bin them in 1000 intervals of equal log-spacing. $idx$ shows in which of the 1000 bins the frequency belongs to and $logbins$ the boundaries of the 1000 bins.

In [3]:
f=jl.range(3*1e-5, 0.5, step=1e-6)
idx,idx26,logbins_26,logbins,f_filtered = jl.GWBackFinder.binning(f)
Sb1, Sb2, Sb3, Sb4, Sb5, Sb6, Sb7, Sb8, Sb9, Sb10, Sb11, Sb12, Sb13, Sb14, Sb15, Sb16, Sb17, Sb18, Sb19, Sb20, Sb21, Sb22, Sb23, Sb24, Sb25, Sb26 = logbins_26[0],logbins_26[1], logbins_26[2], logbins_26[3], logbins_26[4], logbins_26[5], logbins_26[6], logbins_26[7], logbins_26[8], logbins_26[9], logbins_26[10], logbins_26[11], logbins_26[12], logbins_26[13], logbins_26[14], logbins_26[15], logbins_26[16], logbins_26[17], logbins_26[18], logbins_26[19], logbins_26[20], logbins_26[21], logbins_26[22], logbins_26[23], logbins_26[24], logbins_26[25]

### generate training data using the julia package

In [4]:
gw_total_list=[]
for i in tqdm(range((1))):
    Data_total = jl.GWBackFinder.model_train_data(np.array(z[i,:]), f, idx,
                                                  f_filtered,logbins, Sb1, Sb2, Sb3, Sb4, Sb5, Sb6, Sb7, Sb8, Sb9, Sb10, 
                                                  Sb11, Sb12, Sb13, Sb14, Sb15, Sb16, Sb17, Sb18, Sb19, Sb20, Sb21, Sb22, Sb23, Sb24, Sb25)

  0%|          | 0/1 [00:00<?, ?it/s]

: 

In [12]:
 Data_total 

0.003000000000000001

In [7]:
logbins[0]

0.003000000000000001

In [4]:
gw_total_list=[]
for i in tqdm(range((1))):
    Data_total = jl.GWBackFinder.model_train_data(np.array(z[i,:]), f, idx,
                                                  f_filtered,logbins, Sb1, Sb2, Sb3, Sb4, Sb5, Sb6, Sb7, Sb8, Sb9, Sb10, 
                                                  Sb11, Sb12, Sb13, Sb14, Sb15, Sb16, Sb17, Sb18, Sb19, Sb20, Sb21, Sb22, Sb23, Sb24, Sb25)
    gw_total_list.append(np.array(Data_total))

  0%|          | 0/1000 [00:00<?, ?it/s]

### convert to tensor 

In [ ]:
gw_total=torch.tensor(np.array(gw_total_list) , dtype=torch.float32)
thetas=torch.tensor(z[0:len(z)])
# %%

### check shapes

In [ ]:
print(gw_total.shape)
print(thetas.shape)


### train and save inference in the file train_200.pkl

In [ ]:
GW_train.train(thetas=thetas, gw_total=gw_total, prior=custom_prior, resume_training=False, validation_fraction=0.2, 
          learning_rate=1e-4, show_train_summary=True, max_num_epochs=200, 
          path_saved=None, path_inference="/data/users/Androniki/", name_file="train_200.pkl", model_type="nsf", hidden_features=64, num_transforms=3)


### get posterior from the train_200.pkl and save it in posterior.pkl

In [ ]:
GW_train.get_posterior("/data/users/Androniki/train_200.pkl","posterior.pkl")